<a href="https://colab.research.google.com/github/partizanos/computational_finance/blob/master/TP12_Comp_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Series 12
Introduction to Computational Finance
DIMITRIS PROIOS

## Bonds


### Using the same bond as last week, and assuming the required yield is 10%: A bond with 3 years maturity, a face value of 100$ and coupons of 10% paid semi-annually:

|time [years] | coupon [CHF] 
|---|---| 
| 0.5 | 5
| 1 | 5
| 1.5 | 5
| 2 | 5
|2.5 | 5
| 3 | 5


### What is the present value of the bond ?

**Answer**

Value of bonds can be calculated with this formula:
$B = (\sum c e^{rt}) + P e^{rt}$

By replacing with the data of our table the formula becomes:

$B = 5 e^{-0.1 * 0.5} + 5 e^{-0.1* 1} +5 e^{-0.1 * 1.5} +5 e^{-0.1* 2} +5 e^{-0.1 * 2.5}  + 105 e^{-0.1 * 3}$


In [1]:
import numpy as np
import math 
discount = lambda t, rate: np.power(np.e, (-t * rate))

total_present_value = lambda face_value, coupons, periods, rate: sum(
    [coupon * discount(t, rate) for t, coupon in zip(periods,coupons)]
)

face_value = 100
coupons =  [ 5, 5, 5, 5, 5, 105]
periods = np.linspace(0.5, 3, 6).tolist()
y = 0.1

present_value = total_present_value(face_value, coupons, periods, y)
print("The bond value is: ", present_value)

The bond value is:  99.35744494713597


### Compute the duration and the convexity of the bond.

**Answer**

#### Duration:

$D =\frac{\sum_{i=1}^{n} t_i c_i e^{-yt_i}}{B}  $ 

Duration also measures the price sensitivity to yield, the rate of change of price with respect to yield or the percentage change in price for a parallel shift in yields


In [2]:
D= sum(
    [periods[i]*coupons[i]* math.exp((-y)*periods[i]  ) 
    for i in range(len(periods))]
)/present_value

print("The Bond duration is", D ) 


The Bond duration is 2.6634906669483684



#### Convexity:

C =$ \frac{\partial ^2  B}{\partial y^2} = \sum_{t=1}^n  c_i t_i^2 e^{-yt_i}$


B: bond price

t: period

y: yield rate

c: coupon value

Convexity is a measure of the curvature, or the degree of the curve, in the relationship between bond prices and bond yields. So when yield change there is a change in the price of the bond

In [3]:
convexity = sum([
    coupons[i]*periods[i]**2 *math.exp((-y)*periods[i]) 
    for i in range(len(periods)) 
])

print("Convexity",convexity ) 

Convexity 756.1815466823521


### Estimate (with and without convexity) what would be the effect of a x% parallel shift of the yield curve when x equals 0.1% and 1%.

**Answer**

There are trhee ways to estimate the price change of a bond depending on the yield difference :
#### 1st way without convexity using duration:

$ \frac{\delta  B}{B}= - D \delta y$

$ \delta  B= -B D \delta y$


In [4]:
delta_B = lambda B,D,Dy : B* (-D) * Dy
Dy = 0.1 - 0.001
print(present_value,D,Dy )
print(" - new y: 0.1%:", delta_B( present_value,D,Dy ))
print("New price: ", present_value - delta_B( present_value,D,Dy ))
print()
Dy = 0.1 - 0.01
print(present_value,D,Dy )
print(" - new y: 1  %:", delta_B(  present_value,D,Dy  ))
print("New price: ", present_value - delta_B( present_value,D,Dy ))


99.35744494713597 2.6634906669483684 0.099
 - new y: 0.1%: -26.19912510354477
New price:  125.55657005068075

99.35744494713597 2.6634906669483684 0.09000000000000001
 - new y: 1  %: -23.817386457767974
New price:  123.17483140490395


### Recalculation using face value

In [6]:
rate = 0.001

pv = total_present_value(face_value, coupons, periods, rate)
print("total coupon adjusted 0.1%: ", pv)

rate = 0.01

pv = total_present_value(face_value, coupons, periods, rate)
print("total coupon adjusted   1%: ", pv)

total coupon adjusted 0.1%:  129.6480063794294
total coupon adjusted   1%:  126.52519521199336


#### With convexity 

C =$ \frac{\partial^2  B}{\partial y^2} $

= > $ \partial^2  B = \partial y^2 C$



$\delta B = B [ \frac{C}{2} (\delta t)^2 - D \delta r ] $
source: https://en.wikipedia.org/wiki/Bond_convexity

In [15]:
#Convexity adjustment
print("Convexity",convexity )
# conv_adj = lambda C, Dy : convexity * Dy**2
conv_adj = lambda B, C, dy : (C/2)  * Dy**2 - D *dy

Dy = 0.1 - 0.001

print(" - Dy:           ",Dy,)
print(" - new y: 0.1%:", conv_adj(present_value, convexity,Dy))
print()
Dy = 0.1 - 0.01
print(" - Dy:         ",Dy,)
print(" - new y: 1  %: ", conv_adj(present_value, convexity,Dy))

Convexity 756.1815466823521
 - Dy:            0.099
 - new y: 0.1%: 3.4419820934889778

 - Dy:          0.09000000000000001
 - new y: 1  %:  2.822821104038173


## Compare your answer to the exact values.

The results are not the same.


## If computing the exact value is so simple, why bothering using an approximation through
duration ?

**Answer**

Asing an approximation through
duration can be useful for many data.
Unfortunately the 3 ways do not converge always.

The most reliable is to recalculate the coupons each time but it is also the most ccomopuationaly expensive way so in big time series we might want to stay on convexity or duration based calculations.